In [496]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import datetime
from datetime import timedelta

In [497]:
data = requests.get('https://min-api.cryptocompare.com/data/v2/histoday?fsym='+coin_name+'&tsym=USD&limit=1095')

df = pd.DataFrame.from_dict(data.json()['Data']['Data'])


{'Response': 'Error',
 'Message': 'CCCAGG market does not exist for this coin pair (DOL-USD)',
 'HasWarning': False,
 'Type': 1,
 'RateLimit': {},
 'Data': {},
 'Cooldown': 0}

In [498]:
df['time'] = pd.to_datetime(df['time'], 
                                  unit='s')
start_date = datetime.datetime(2021,1,1)
end_date= df['time'].iloc[-1]
new_df = df[df['time'].between(start_date, end_date)]
new_df = new_df.reset_index(drop=True)

In [499]:
start_dates = pd.DataFrame(pd.date_range(start_date,end_date, 
              freq='MS').strftime("%Y-%m-%d").tolist(), columns={'start_date'})
start_dates['start_date'] = pd.to_datetime(start_dates['start_date'])


end_dates = pd.DataFrame()
end_dates['end_date'] = start_dates['start_date'] - pd.DateOffset(days=1)
end_dates = end_dates.drop(end_dates.index[0])

if (end_dates['end_date'].iloc[-1] != new_df['time'].iloc[-1]):
    end_dates.loc[len(end_dates)] = new_df['time'].iloc[-1]

end_dates = end_dates.reset_index(drop=True)


In [500]:
T_minus_1_day = pd.DataFrame()
T_minus_1_day['date'] = end_dates['end_date'] - pd.DateOffset(days=1)

T_minus_1_day_CP_list = []
for i in range(len(T_minus_1_day)):
    T_minus_1_day_CP_list.append(new_df.loc[new_df['time'] == T_minus_1_day['date'].iloc[i], 'close'].iloc[0])

T_minus_1_day_CP = pd.DataFrame(T_minus_1_day_CP_list, columns={'T_minus_1_day_CP'})


T_minus_30_day = pd.DataFrame()
T_minus_30_day['date'] = end_dates['end_date'] - pd.DateOffset(days=30)


T_minus_30_day_CP_list = []
for i in range(len(T_minus_30_day)):
    T_minus_30_day_CP_list.append(new_df.loc[new_df['time'] == T_minus_30_day['date'].iloc[i], 'close'].iloc[0])

T_minus_30_day_CP = pd.DataFrame(T_minus_30_day_CP_list, columns={'T_minus_30_day_CP'})



In [501]:
Returns = T_minus_1_day_CP['T_minus_1_day_CP'] / T_minus_30_day_CP['T_minus_30_day_CP']
Returns = pd.DataFrame(Returns,columns={'Returns'}) -1
Returns

,Returns
0,0.167712
1,0.348572
2,0.184274
3,-0.088653
4,-0.383461
5,-0.037259
6,0.259031
7,0.178641
8,-0.119167
9,0.285015


In [502]:
T_day_OP_list = []
for i in range(len(T_minus_30_day)):
    T_day_OP_list.append(new_df.loc[new_df['time'] == end_dates['end_date'].iloc[i], 'open'].iloc[0])

T_day_OP = pd.DataFrame(T_day_OP_list, columns={'T_day_OP'})
T_day_OP

,T_day_OP
0,34317.82
1,46191.39
2,58781.54
3,53581.58
4,35662.59
5,35901.41
6,42231.83
7,46993.31
8,41539.43
9,61891.91
